<a href="https://colab.research.google.com/github/ogigo/text-classification/blob/main/word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
!mkdir -p ~/.kaggle

In [3]:
api_token = {"username":"shahedshoyab","key":"1aa2d9748604f09744dac84eb27b107f"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:json.dump(api_token, file)

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download clmentbisaillon/fake-and-real-news-dataset

 93% 38.0M/41.0M [00:00<00:00, 222MB/s] 
100% 41.0M/41.0M [00:00<00:00, 205MB/s]


In [6]:
!unzip fake-and-real-news-dataset.zip

Archive:  fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [7]:
import pandas as pd


In [8]:
true = pd.read_csv("/content/True.csv")
fake = pd.read_csv("/content/Fake.csv")

In [9]:
# removing unnecessary colums
true = true.drop(['subject', 'date', 'title'], axis=1)
fake = fake.drop(['subject', 'date', 'title'], axis=1)

In [10]:
# labelling the data
true['label'] = 1
fake['label'] = 0

In [11]:
# concatenating both dataframes
df = pd.concat([true, fake])
# shuffling the data
df = df.sample(frac = 1)
# resetting the index values
df.reset_index(inplace=True, drop=True)

In [12]:
# removing duplicate values
df.drop_duplicates(subset=None, keep='first', inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.head()

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

In [15]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
df['text']=df['text'].apply(denoise_text)

In [ ]:
df.head()

In [18]:
x_train,x_test,y_train,y_test = train_test_split(df.text,df.label,random_state = 0)

In [19]:
max_features=10000
max_len=300

In [20]:
tokenizer=text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)

In [21]:
tokenizer_train=tokenizer.texts_to_sequences(x_train)
tokenizer_test=tokenizer.texts_to_sequences(x_test)

In [ ]:
len(tokenizer_train)

In [23]:
x_train=sequence.pad_sequences(tokenizer_train,maxlen=max_len)
x_test=sequence.pad_sequences(tokenizer_test,maxlen=max_len)

In [24]:
import os

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

In [26]:
path_to_glove_file = os.path.join("/content/glove.6B.100d.txt")

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [31]:
word_index=tokenizer.word_index
number_words=len(word_index)+2
embedding_dim=100
hits=0
misses=0

In [38]:
embedding_matrix=np.zeros((number_words,embedding_dim))
for word,i in word_index.items():
  embedding_vector=embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i]=embedding_vector
    hits=hits+1
  else:
    misses=misses+1

print("Converted %d words (%d misses)" % (hits, misses))


Converted 70537 words (44745 misses)


In [ ]:
embedding_matrix

In [42]:
embedding=Embedding(
    input_dim=number_words,
    output_dim=embedding_dim,
    embeddings_initializer=keras.initializers.constant(embedding_matrix),
    trainable=False
)

In [43]:
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(embedding)
#LSTM 
model.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(units=64 , recurrent_dropout = 0.1 , dropout = 0.1))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [44]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience = 2, 
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

In [45]:
history = model.fit(x_train, y_train,
                    batch_size = 256 , 
                    validation_data = (x_test,y_test) , 
                    epochs = 1 , 
                    callbacks = [learning_rate_reduction])

114/114 [==============================] - 578s 5s/step - loss: 0.1873 - accuracy: 0.9201 - val_loss: 0.1071 - val_accuracy: 0.9591 - lr: 0.0100
